In [19]:
import numpy as np
import pandas as pd
from datetime               import datetime
from google.cloud           import bigquery
from google.oauth2          import service_account
from google.cloud.bigquery  import Client

In [20]:
credentials = service_account.Credentials.from_service_account_file('nowheredata-9b61624b5615.json')
client = bigquery.Client(credentials=credentials, project='nowheredata')

In [21]:
# def get_data(sql_file, var):  
#     if var == True:
#         dados = client.query(sql_file)
#         result = dados.result()
#         df = result.to_dataframe()
#         return df
#     else:
#         with open(sql_file, 'r') as f:
#             sql = f.read()
#         dados = client.query(sql)
#         result = dados.result()
#         df = result.to_dataframe()
#         return df

In [22]:
# max_rows = 1048576

# def save_to_multiple_sheets(df, writer, sheet_name_prefix):
#     num_chunks = -(-len(df) // (max_rows - 1))
#     for i in range(num_chunks):
#         start_idx = i * (max_rows - 1)
#         end_idx = (i + 1) * (max_rows - 1)
#         df_chunk = df.iloc[start_idx:end_idx]
#         df_chunk.to_excel(writer, sheet_name=f"{sheet_name_prefix}_{i+1}", index=False)

In [23]:
project_id = 'nowheredata'
dataset = 'raw_tables'
table_name = 'fact_orders'

In [24]:
df = pd.read_csv('orders.csv', sep=';', encoding='utf-8-sig', decimal=",")

In [25]:
df

,order_id,order_date,user_id,product_id,revenue
0,8213351680,02/08/2024,75361105,32189,3110.96
1,7536474509,29/10/2024,43580092,60146,645.26
2,5205005404,07/03/2024,75361105,50781,844.18
3,8213351680,02/08/2024,75361105,21032,3110.96
4,2673322804,28/02/2024,75361105,44933,3380.05
...,...,...,...,...,...
1646,3043278459,27/05/2024,37404863,23393,7701.88
1647,3043278459,27/05/2024,37404863,78396,7701.88
1648,3043278459,27/05/2024,37404863,49121,7701.88
1649,3043278459,27/05/2024,37404863,53678,7701.88


In [26]:
df.dtypes

order_id        int64
order_date     object
user_id         int64
product_id      int64
revenue       float64
dtype: object

In [27]:
conversion_dict = {
    'order_id': 'int64',
    #'order_date': 'date',
    'user_id': 'int64',
    'product_id': 'int64',
    'revenue': 'float64'
}

df = df.astype(conversion_dict)
df["order_date"] = pd.to_datetime(
    df["order_date"].astype(str).str.strip(),
    format="%d/%m/%Y",
    errors="coerce"
)

In [28]:
df.dtypes

order_id               int64
order_date    datetime64[ns]
user_id                int64
product_id             int64
revenue              float64
dtype: object

In [29]:
schema = [
    bigquery.SchemaField('order_id','INT64', mode='NULLABLE'),
    bigquery.SchemaField('order_date','DATE', mode='NULLABLE'),
    bigquery.SchemaField('user_id','INT64', mode='NULLABLE'),
    bigquery.SchemaField('product_id','INT64', mode='NULLABLE'),
    bigquery.SchemaField('revenue','FLOAT64', mode='NULLABLE'),
]

In [30]:
table_ref = client.dataset(dataset).table(table_name)
try:
    table = client.get_table(table_ref)
    print('A tabela já existe. Os novos dados serão carregados na tabela.')
except:
    print('A tabela não existe. Será criada agora.')
    table = bigquery.Table(table_ref)
    table = client.create_table(table)
    print('A tabela foi criada com sucesso.')

# Carrega os novos dados na tabela existente
job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE', schema=schema)
job = client.load_table_from_dataframe(df.drop_duplicates(), table_ref, job_config=job_config)
job.result()
print('Os novos dados foram carregados com sucesso na tabela existente.')

A tabela já existe. Os novos dados serão carregados na tabela.
Os novos dados foram carregados com sucesso na tabela existente.
